# Kubeflow Pipeline that Uses Vertex AI to train and deploy an Image Classification model

Demonstrates how to use Google Kubeflow Components to do the following.

1. Create a Vertex AI Image Dataset
2. Run an Image Classification Training job with AutoML
3. Create an Enpoint
4. Deploy the Model to the Endpoint

In [ ]:
!pip install --upgrade google-cloud-aiplatform google-cloud-pipeline-components kfp

In [ ]:
# Change these values to match your project setup
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

REGION = "us-central1" # @param {type:"string"}

# An existing bucket to store pipeline artifacts.
BUCKET_NAME = "basic-kf-pipelines-mlops-dar" # @param {type:"string"}

# GCP settings
GCS_BUCKET = "gs://mlops-data-sources" # @param {type:"string"}
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline-artifacts"


In [ ]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components.v1.dataset import ImageDatasetCreateOp
from google_cloud_pipeline_components.v1.automl.training_job import AutoMLImageTrainingJobRunOp
from google_cloud_pipeline_components.v1.endpoint import EndpointCreateOp, ModelDeployOp

project_id = PROJECT_ID
pipeline_root_path = PIPELINE_ROOT

# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="mushrooms-classification-pipeline",
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str):
    ds_op = ImageDatasetCreateOp(
        project=project_id,
        display_name="mushrooms",
        gcs_source="gs://mushrooms-dar/mushrooms.csv",
        import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
    )

    training_job_run_op = AutoMLImageTrainingJobRunOp(
        project=project_id,
        display_name="mushrooms-training",
        prediction_type="classification",
        model_type="CLOUD",
        dataset=ds_op.outputs["dataset"],
        model_display_name="mushrooms-classification-model",
        training_fraction_split=0.6,
        validation_fraction_split=0.2,
        test_fraction_split=0.2,
        budget_milli_node_hours=8000,
    )


    create_endpoint_op = EndpointCreateOp(
        project=project_id,
        display_name = "mushrooms-endpoint",
    )

    model_deploy_op = ModelDeployOp(
        model=training_job_run_op.outputs["model"],
        endpoint=create_endpoint_op.outputs['endpoint'],
        automatic_resources_min_replica_count=1,
        automatic_resources_max_replica_count=1,
    )

In [ ]:
from kfp import compiler

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path='mushrooms-image-classification-pipeline.yaml'
)

In [ ]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(project=PROJECT_ID, location=REGION)

# Prepare the pipeline job
pipeline_job = aip.PipelineJob(
    display_name="mushrooms-image-classification-pipeline",
    template_path="mushrooms-image-classification-pipeline.yaml",
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': PROJECT_ID
    },
    enable_caching=False
)

pipeline_job.submit()